In [50]:
import json
import pandas as pd
import requests

In [13]:
# Read gloassary data from json
with open('WLASL_v0.3.json', 'r') as f:
    data = json.load(f)

In [42]:
# Create a DataFrame with information on the videos
dfs = []
for sign in data :
    word = sign.get('gloss')
    df = pd.DataFrame(sign.get('instances'))
    df['word'] = word
    dfs.append(df)

video_information = pd.concat(dfs).reset_index(drop = True)
video_information.head()

,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,word
0,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,book
1,"[190, 25, 489, 370]",25,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65225,book
2,"[262, 1, 652, 480]",25,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0,68011,book
3,"[123, 19, 516, 358]",25,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0,68208,book
4,"[95, 0, 1180, 720]",25,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0,68012,book


In [45]:
# Find which videos are mp4 links
video_information['mp4'] = video_information['url'].apply(lambda x: x.split('.')[-1]) == 'mp4'
video_mp4 = video_information.groupby('mp4').get_group(True)
video_mp4 = video_mp4[(video_mp4['word'].isin(video_mp4['word'].value_counts().head(50).index.to_list()))]

In [46]:
video_mp4

,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,word,mp4
40,"[551, 68, 1350, 1080]",25,-1,1,0,115,aslbrick,val,http://aslbricks.org/New/ASL-Videos/drink.mp4,0,69302,drink,True
41,"[153, 11, 488, 370]",25,-1,1,1,94,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65539,drink,True
51,"[196, 15, 521, 414]",25,-1,1,11,36,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,17710,drink,True
56,"[186, 63, 551, 400]",25,-1,1,16,12,aslsearch,train,http://www.aslsearch.com/signs/videos/drink-al...,0,17733,drink,True
57,"[167, 19, 480, 370]",25,-1,1,17,94,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65540,drink,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7034,"[157, 32, 489, 370]",25,-1,1,9,103,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,1,66434,score,True
7035,"[70, 16, 260, 240]",25,-1,1,10,0,spreadthesign,val,https://media.spreadthesign.com/video/mp4/13/8...,0,49691,score,True
7036,"[82, 18, 199, 192]",25,-1,1,12,10,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/23/2376...,1,49702,score,True
7037,"[241, 38, 495, 400]",25,-1,1,16,41,aslsearch,train,http://www.aslsearch.com/signs/videos/score.mp4,0,49706,score,True


In [55]:
def download_video(url, filename):
    # Send a GET request to the URL to fetch the content
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the file in binary write mode and write the content
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Download successful [{url}].", end = '\r')
    else:
        print("Failed to download video.", end = '\r')

In [60]:
for j, (_, row) in enumerate(video_mp4.iterrows()) :
    
    print(j)
    filename = f"video_data/{row['word']}/{row['video_id']}.mp4"
    download_video(row['url'], filename)

0
1ownload successful [http://aslbricks.org/New/ASL-Videos/drink.mp4].
2ailed to download video.
3ownload successful [https://s3-us-west-1.amazonaws.com/files.startasl.com/asldictionary/drink.mp4].
4ownload successful [http://www.aslsearch.com/signs/videos/drink-alcohol.mp4].
5ownload successful [https://aslsignbank.haskins.yale.edu/dictionary/protected_media/glossvideo/ASL/DR/DRINK-120.mp4].
6ownload successful [http://www.aslsearch.com/signs/videos/drink.mp4].
7ownload successful [https://signstock.blob.core.windows.net/signschool/videos/db_uploads/SignSchool%20Drink%20(correct)-0UzEF7ru0sQ.mp4].
8ownload successful [https://signstock.blob.core.windows.net/signschool/videos/db_uploads/SignSchool%20Drink-L6hHRIQzWBo.mp4].
9ownload successful [https://media.asldeafined.com/vocabulary/1467773819.9825.mp4].
10iled to download video.
11iled to download video.
12iled to download video.
13iled to download video.
14iled to download video.
15wnload successful [https://media.spreadthesign.com/